# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab XX**: 03

**Fecha**: 14/02/2025

**Nombre del Estudiante**: Alejandro Samuel Romero Mora

**Profesor**: Pablo Camarillo Ramirez

#### Encontrar la instalación de PySpark

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Map-Reduce-Examples") \
    .master("spark://a45e80e2de60:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/14 06:48:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Example 1: Convert Temperatures from Celsius to Fahrenheit
This example demonstrates how to use `map()` to transform data.

###Creamos RDD

In [3]:
# Create an RDD of temperatures in Celsius
posts = ["Learning #BigData with #PySpark is fun! #AI",
         "#AI is transforming the world. #BigData #MachineLearning",
         "I love #PySpark and #BigData. #AI #DataScience",
         "#DataScience and #AI are the future. #BigData",
         "#PySpark is awesome! #BigData #AI"]

posts_rdd = sc.parallelize(posts)

#### Map Phase

In [4]:
def get_hashtags(post):
    return [word for word in post.split() if word.startswith('#')]

hashtags_rdd = posts_rdd.flatMap(get_hashtags)


In [5]:
print(hashtags_rdd.collect())


['#BigData', '#PySpark', '#AI', '#AI', '#BigData', '#MachineLearning', '#PySpark', '#BigData.', '#AI', '#DataScience', '#DataScience', '#AI', '#BigData', '#PySpark', '#BigData', '#AI']


In [6]:
tuples_hashtag_rdd = hashtags_rdd.map(lambda x: (x,1))
tuples_hashtag_rdd.collect()

[('#BigData', 1),
 ('#PySpark', 1),
 ('#AI', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#MachineLearning', 1),
 ('#PySpark', 1),
 ('#BigData.', 1),
 ('#AI', 1),
 ('#DataScience', 1),
 ('#DataScience', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#PySpark', 1),
 ('#BigData', 1),
 ('#AI', 1)]

In [36]:
#Tenemos que hacer cast a diccionario para que salgan los datos de manera limpia
print("Hashtag counts: ",dict(tuples_hashtag_rdd.countByKey()))

Hashtag counts:  {'#BigData': 4, '#PySpark': 3, '#AI': 5, '#MachineLearning': 1, '#BigData.': 1, '#DataScience': 2}


In [35]:
# Mapeamos cada elemento para obtener un par (longitud del hashtag, hashtag)
grouped_by_lenght = tuples_hashtag_rdd.map(lambda x: (len(x[0]), x[0]))

#Obtenemos la longitud del hashtag y el hashtag en sí, para luego hacer una lista de las ocurrencias de cada uno (por longitud)
print("Datos mapeados:", grouped_by_lenght.collect())

#Agrupamos por llave y convertimos los valores agrupados en listas
grouped_by_lenght = grouped_by_lenght.groupByKey().mapValues(list)
print("Datos mapeados:", grouped_by_lenght.collect())

for length, tag_list in grouped_by_lenght.collect():
    print(f"Length {length}: {tag_list}")


Datos mapeados: [(8, '#BigData'), (8, '#PySpark'), (3, '#AI'), (3, '#AI'), (8, '#BigData'), (16, '#MachineLearning'), (8, '#PySpark'), (9, '#BigData.'), (3, '#AI'), (12, '#DataScience'), (12, '#DataScience'), (3, '#AI'), (8, '#BigData'), (8, '#PySpark'), (8, '#BigData'), (3, '#AI')]
Datos mapeados: [(8, ['#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData']), (16, ['#MachineLearning']), (12, ['#DataScience', '#DataScience']), (3, ['#AI', '#AI', '#AI', '#AI', '#AI']), (9, ['#BigData.'])]
Length 8: ['#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData']
Length 16: ['#MachineLearning']
Length 12: ['#DataScience', '#DataScience']
Length 3: ['#AI', '#AI', '#AI', '#AI', '#AI']
Length 9: ['#BigData.']
